# Notebook Setup

In [1]:
if 'google.colab' in str(get_ipython()):
  IN_COLLAB = True
else:
  IN_COLLAB = False

#TODO: CHANGE THIS BASED ON YOUR OWN LOCAL SETTINGS
MY_HOME_ABS_PATH = "/content/drive/MyDrive/W210/co2-flux-hourly-gpp-modeling"

if IN_COLLAB:
  from google.colab import drive
  drive.mount('/content/drive/')

Mounted at /content/drive/


## Import Modules

In [ ]:
# install required modules quietly
required_packages = ['geopandas','azure-storage-blob']

for p in required_packages: 
  try:
      __import__(p)
  except ImportError:
      %pip install {p} --quiet

In [ ]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

import pandas as pd
from io import BytesIO

# Load locale custome modules
import sys
if IN_COLLAB:
  os.chdir(MY_HOME_ABS_PATH)
  sys.path.insert(0,os.path.abspath("./code/src/tools"))
else:
  sys.path.append(os.path.abspath("./code/src/tools"))

from CloudIO.AzStorageClient import AzStorageClient
from data_pipeline_lib import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

## Define Local File System Constants

In [ ]:
root_dir =  MY_HOME_ABS_PATH
tmp_dir =  root_dir + os.sep + '.tmp'
raw_data_dir = tmp_dir
data_dir = root_dir + os.sep + 'data'
cred_dir = root_dir + os.sep + '.cred'
az_cred_file = cred_dir + os.sep + 'azblobcred.json'

if IN_COLLAB:
  raw_data_dir = "/content/drive/MyDrive/CO2_flux_gpp_modeling/DS_capstone_23Spring_CO2/Data/half_hourly_data"

In [ ]:
train_sites = ['US-NR1', 'US-Me2', 'US-ARM', 'US-Vcp']
valid_sites = ["US-Vcp"]
test_sites = ["US-GLE"]
selected_sites =  train_sites + valid_sites + test_sites

# Load raw dataset from Azure Storage Blob

In [ ]:
# Define target dataset
container = "baseline-data"
ext = "parquet"
ver = "0"
tag = "mc_raw"
blob_name_base = f"baseline_all_v_{ver}"
data_blob_name = f"{blob_name_base}_{tag}.{ext}"

full_blob_name = 'baseline_all_v_1-i-knn_raw.parquet'

tft_data_dir = tmp_dir + os.sep + 'data' + os.sep + 'gpp'

download_data = False

In [ ]:
def load_data(container, data_blob_name, local_file_path):
  if not (os.path.exists(local_file_path)):
    if not (os.path.exists(tmp_dir)):
      os.mkdir(tmp_dir)
    if not (os.path.exists(tmp_dir + os.sep + 'data')):
      os.mkdir(tmp_dir + os.sep + 'data')
      os.mkdir(tmp_dir + os.sep + 'data' + os.sep + 'gpp')
    print(f"Loading dataset from {container}/{data_blob_name}...")
    # Initialized Azure Storage Client
    azStorageClient = AzStorageClient(az_cred_file)
    file_stream = azStorageClient.downloadBlob2Stream(container, data_blob_name)
    data_df = pd.read_parquet(file_stream, engine='pyarrow')
    data_df.to_csv(local_file_path)
  else:
    print(f"Loading dataset from {local_file_path}...")
    data_df = pd.read_csv(local_file_path)

  return data_df

In [ ]:
if download_data:
  # Load full dataset
  full_local_file = tft_data_dir + os.sep + 'full.csv'

  full_df = load_data(container, full_blob_name, full_local_file)

  print(f"Data size: {full_df.shape}")
  print(full_df['site_id'].unique())
  print("data peak:")
  display(full_df.head())
  

In [ ]:
# import gc
# del full_df
# gc.collect()

# Run TensortFlow TFT (Fix-Param, Test)

https://www.tensorflow.org/guide/migrate/upgrade

In [3]:
import os
os.chdir(MY_HOME_ABS_PATH + "/code/src/tools/tft/")

In [4]:
!pip install tensorflow-gpu
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tensorflow-gpu
  Running setup.py clean for tensorflow-gpu
Failed to build tensorflow-gpu
  error: subprocess-exited-with-error
  
  × Running setup.py install for tensorflow-gpu did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Running setup.py install for tensorflow-gpu ... error
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> tensorflow-gpu

note: This is an issue with the package mentioned above, not pip.
hint: See above for outpu

In [5]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.11.0
/device:GPU:0
Num GPUs Available:  1


In [ ]:
# Update tesnort flow libs
# !tf_upgrade_v2 \
#     --intree libs/ \
#     --outtree libs_v2/ \
#     --reportfile tree_report.txt

In [6]:
tf.compat.v1.experimental.output_all_intermediates(True)

In [ ]:
%run script_train_fixed_params.py gpp ../../../../.tmp yes yes

Using output folder ../../../../.tmp


Selecting GPU ID=0
*** Training from defined parameters for gpp ***
Loading & splitting data...
Formatting train-valid-test splits.
Raw size: (865920, 50) from ['CA-Cbo' 'CH-Lae' 'ES-LJu' 'ES-LM2' 'FI-Hyy' 'FR-Lam' 'FR-Pue' 'IT-Lav'
 'US-AR1' 'US-ARM' 'US-GLE' 'US-NR1' 'US-SRM' 'US-Seg' 'US-Ton' 'US-UMB'
 'US-Var' 'US-Vcp' 'US-Wkg']
Setting scalers with training data...
IDs:['CH-Lae', 'ES-LJu', 'FI-Hyy', 'FR-Pue', 'IT-Lav', 'US-ARM', 'US-NR1', 'US-SRM', 'US-Ton', 'US-UMB', 'US-Var', 'US-Vcp', 'US-Wkg']
Real number input: ['TA_ERA', 'SW_IN_ERA', 'LW_IN_ERA', 'VPD_ERA', 'P_ERA', 'PA_ERA', 'day', 'hour', 'lat', 'long', 'TA_ERA', 'SW_IN_ERA', 'LW_IN_ERA', 'VPD_ERA', 'P_ERA', 'PA_ERA', 'day', 'hour', 'lat', 'long'].
Setting scalers with training data...
Categorical input: ['year', 'month', 'IGBP', 'koppen_main', 'koppen_sub']
Train size: (631032, 50) from ['CH-Lae' 'ES-LJu' 'FI-Hyy' 'FR-Pue' 'IT-Lav' 'US-ARM' 'US-NR1' 'US-SRM'
 'US-Ton' 'US-UMB' 'US-Var' 'US-Vcp' 'US-Wkg']
Valid size: (8589

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 192, 26)]    0           []                               
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['input_1[0][0]']                
 sformer/strided_slice_1 (Tenso                                                                   
 rFlowOpLayer)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 21)]   0           ['input_1[0][0]']                
 sformer/strided_slice (TensorF                                                               

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


                                                                                                  
 dense_30 (Dense)               (None, 1, 5)         30          ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_37[0][0]']  
                                                                                                  
 dense_34 (Dense)               (None, 1, 5)         30          ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_38[0][0]']  
                                                                                                  
 dense_38 (Dense)               (None, 1, 5)         30          ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_39[0][0]']  
                                                                                                  
 dense_42 

/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2333: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


100/100 [==============================] - 34s 341ms/sample - loss: 5.6535 - val_loss: 3.3941
Cannot load from ../../../../.tmp/saved_models/gpp/fixed/tmp, skipping ...
Using cached validation data
Optimal model found, updating
*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.1
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 100.0
# minibatch_size = 128
# model_folder = ../../../../.tmp/saved_models/gpp/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('site_id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('datetime', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('GPP_NT_VUT_REF', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('TA_ERA', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('SW_IN_ERA', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPU

# Model Evaulation Visualization